In [1]:
import spacy
import zh_core_web_sm
import  pkuseg
import jieba
import jieba.posseg
import pycantonese as pc

In [2]:
import pynlpir

In [3]:
nlp = zh_core_web_sm.load()

Building prefix dict from the default dictionary ...
Loading model from cache /tmp/jieba.cache
Loading model cost 0.811 seconds.
Prefix dict has been built successfully.


In [4]:
import re
import numpy as np
import pandas as pd
import re as re
from math import sqrt
from joblib import Parallel, delayed

In [5]:
df = pd.read_excel(r'All.xlsx')
df = pd.DataFrame(df)


In [6]:
dfchinese = pd.DataFrame(columns=['Text'])
for i in range(len(df)):
    x = re.sub("[a-z]|[0-9]|M","", df['Text'].iloc[i])
    y = re.sub("/","", x)
    z = re.sub(" ","", y)
    dfchinese = dfchinese.append({'Text': z}, ignore_index=True)


In [7]:
print('What sentence we will analyse','\n', dfchinese['Text'].iloc[2])

What sentence we will analyse 
 每天早上，媽媽都會為我準備早餐，並且送我到學校。然後，她就去運動場跑步。放學後，當我做功課時，有不明白的地方，媽媽便像老師一樣，坐到我身邊，循循善誘地指導我，令我茅塞頓開。


In [8]:
#pkuseg segmentation but no PoS
seg  =  pkuseg.pkuseg () 
a = dfchinese['Text'].iloc[2]
text  =  seg.cut(a)
text

['每天',
 '早上',
 '，',
 '媽媽',
 '都',
 '會為',
 '我',
 '準備',
 '早餐',
 '，',
 '並且',
 '送',
 '我',
 '到',
 '學校',
 '。',
 '然後',
 '，',
 '她',
 '就',
 '去',
 '運動場',
 '跑步',
 '。',
 '放學',
 '後',
 '，',
 '當',
 '我',
 '做',
 '功課時',
 '，',
 '有',
 '不',
 '明白',
 '的',
 '地方',
 '，',
 '媽媽',
 '便',
 '像',
 '老師',
 '一',
 '樣',
 '，',
 '坐到',
 '我',
 '身邊',
 '，',
 '循循善誘',
 '地',
 '指導',
 '我',
 '，',
 '令',
 '我',
 '茅塞',
 '頓開',
 '。']

In [9]:
#JIEBA

words = jieba.posseg.cut(a)
c=[]
for word, flag in words:
    c.append('%s %s' % (word, flag))
c

['每天 r',
 '早上 t',
 '， x',
 '媽媽 n',
 '都 d',
 '會 v',
 '為 p',
 '我 r',
 '準備 v',
 '早餐 n',
 '， x',
 '並且 c',
 '送 v',
 '我 r',
 '到 v',
 '學校 n',
 '。 x',
 '然 r',
 '後 nr',
 '， x',
 '她 r',
 '就 d',
 '去 v',
 '運動場 n',
 '跑步 n',
 '。 x',
 '放學 n',
 '後 nr',
 '， x',
 '當 p',
 '我 r',
 '做功 v',
 '課時 t',
 '， x',
 '有 v',
 '不 d',
 '明白 nr',
 '的 uj',
 '地方 n',
 '， x',
 '媽媽 n',
 '便 d',
 '像 v',
 '老師 n',
 '一樣 u',
 '， x',
 '坐到 v',
 '我 r',
 '身邊 s',
 '， x',
 '循循善誘 i',
 '地 uv',
 '指導 v',
 '我 r',
 '， x',
 '令 v',
 '我 r',
 '茅塞 nr',
 '頓 q',
 '開 v',
 '。 x']

In [10]:
#SPACY
nlp = zh_core_web_sm.load()
    
doc = nlp(u'每天早上，媽媽都會為我準備早餐，並且送我到學校。然後，她就去運動場跑步。放學後，當我做功課時，有不明白的地方，媽媽便像老師一樣，坐到我身邊，循循善誘地指導我，令我茅塞頓開。')
for token in doc:
    print(token.text, token.pos_)

每天 ADV
早上 NOUN
， PUNCT
媽媽 NOUN
都 ADV
會為 VERB
我 PRON
準備 NOUN
早餐 ADV
， PUNCT
並且 VERB
送 VERB
我 PRON
到 VERB
學校 VERB
。 PUNCT
然後 ADV
， PUNCT
她 PRON
就 ADV
去 VERB
運 VERB
動場 NOUN
跑步 VERB
。 PUNCT
放學 VERB
後 PART
， PUNCT
當 VERB
我 PRON
做 VERB
功課時 NOUN
， PUNCT
有 VERB
不 ADV
明白 VERB
的 PART
地方 NOUN
， PUNCT
媽媽 NOUN
便 ADV
像 ADP
老師 NOUN
一 NUM
樣 NUM
， PUNCT
坐到 VERB
我 PRON
身邊 NOUN
， PUNCT
循循善誘 ADV
地 PART
指導 VERB
我 PRON
， PUNCT
令 VERB
我 PRON
茅塞 ADV
頓開 VERB
。 PUNCT


In [11]:
from rakutenma import RakutenMA
rma = RakutenMA()
# Initialize a RakutenMA instance with a pre-trained model
rma = RakutenMA(phi=1024, c=0.007812)  # Specify hyperparameter for SCW (for demonstration purpose)
rma.load("model_ja.json")

# Set the feature hash function (15bit)
rma.hash_func = rma.create_hash_func(15)

# Tokenize one sample sentence
print(rma.tokenize('每天早上，媽媽都會為我準備早餐，並且送我到學校。然後，她就去運動場跑步。放學後，當我做功課時，有不明白的地方，媽媽便像老師一樣，坐到我身邊，循循善誘地指導我，令我茅塞頓開。'));




[['每天', 'N-nc'], ['早上', 'N-nc'], ['，', 'M-c'], ['媽媽', 'N-nc'], ['都會', 'N-nc'], ['為我', 'N-nc'], ['準備', 'N-nc'], ['早餐', 'A-c'], ['，', 'M-c'], ['並且', 'N-nc'], ['送我', 'N-nc'], ['到', 'N-nc'], ['學校', 'N-nc'], ['。', 'M-p'], ['然後', 'N-nc'], ['，', 'M-c'], ['她', 'N-nc'], ['就去', 'N-nc'], ['運動', 'N-nc'], ['場跑', 'N-nc'], ['步', 'Q-n'], ['。', 'M-p'], ['放學', 'N-nc'], ['後', 'Q-n'], ['，', 'M-c'], ['當我', 'N-nc'], ['做功', 'N-nc'], ['課時', 'N-nc'], ['，', 'M-c'], ['有不', 'N-pn'], ['明白', 'N-nc'], ['的', 'Q-j'], ['地方', 'N-nc'], ['，', 'M-c'], ['媽媽', 'N-nc'], ['便像', 'N-nc'], ['老師', 'N-nc'], ['一樣', 'N-pn'], ['，', 'M-c'], ['坐', 'N-nc'], ['到我', 'N-nc'], ['身邊', 'N-nc'], ['，', 'M-c'], ['循循善', 'N-nc'], ['誘地', 'N-nc'], ['指導', 'N-nc'], ['我', 'Q-n'], ['，', 'M-c'], ['令我', 'N-nc'], ['茅塞', 'N-nc'], ['頓開', 'N-nc'], ['。', 'M-p']]


In [12]:
import thulac   

thu1 = thulac.thulac()  #默认模式
text = thu1.cut('每天早上，媽媽都會為我準備早餐，並且送我到學校。然後，她就去運動場跑步。放學後，當我做功課時，有不明白的地方，媽媽便像老師一樣，坐到我身邊，循循善誘地指導我，令我茅塞頓開。', text=True)  #进行一句话分词
print(text)

Model loaded succeed
每天_r 早上_t ，_w 媽媽_n 都_d 會為_v 我_r 準備_v 早餐_n ，_w 並且_c 送_v 我_r 到_v 學校_n 。_w 然後_a ，_w 她_r 就_d 去_v 運動場_n 跑步_v 。_w 放_v 學後_n ，_w 當_v 我_r 做_v 功課時_id ，_w 有_v 不_d 明白_a 的_u 地方_n ，_w 媽媽_n 便_d 像_v 老師一樣_id ，_w 坐_v 到_v 我_r 身_g 邊_v ，_w 循循善誘_id 地_u 指導_v 我_r ，_w 令_v 我_r 茅塞頓開_id 。_w


In [13]:
import pynlpir
pynlpir.open()

s = '每天早上，媽媽都會為我準備早餐，並且送我到學校。然後，她就去運動場跑步。放學後，當我做功課時，有不明白的地方，媽媽便像老師一樣，坐到我身邊，循循善誘地指導我，令我茅塞頓開。'
pynlpir.segment(s)

[('每天', 'pronoun'),
 ('早上', 'time word'),
 ('，', 'punctuation mark'),
 ('媽', 'noun'),
 ('媽', 'noun'),
 ('都', 'adverb'),
 ('會', 'noun'),
 ('為', 'noun'),
 ('我', 'pronoun'),
 ('準', 'noun'),
 ('備', 'noun'),
 ('早餐', 'noun'),
 ('，', 'punctuation mark'),
 ('並', 'noun'),
 ('且', 'conjunction'),
 ('送', 'verb'),
 ('我', 'pronoun'),
 ('到', 'verb'),
 ('學', 'noun'),
 ('校', 'noun'),
 ('。', 'punctuation mark'),
 ('然', 'pronoun'),
 ('後', 'noun'),
 ('，', 'punctuation mark'),
 ('她', 'pronoun'),
 ('就', 'adverb'),
 ('去', 'verb'),
 ('運', 'noun'),
 ('動', 'noun'),
 ('場', 'noun'),
 ('跑步', 'verb'),
 ('。', 'punctuation mark'),
 ('放', 'verb'),
 ('學', 'noun'),
 ('後', 'noun'),
 ('，', 'punctuation mark'),
 ('當', 'noun'),
 ('我', 'pronoun'),
 ('做功', 'noun'),
 ('課', 'noun'),
 ('時', 'noun'),
 ('，', 'punctuation mark'),
 ('有', 'verb'),
 ('不', 'adverb'),
 ('明白', 'adjective'),
 ('的', 'particle'),
 ('地方', 'noun'),
 ('，', 'punctuation mark'),
 ('媽', 'noun'),
 ('媽', 'noun'),
 ('便', 'adverb'),
 ('像', 'verb'),
 ('老', 'adjective'